### Importing Necessary libraries

In [1]:
import sys

import os
import re
import numpy as np
import pandas as pd
from time import time
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
from transformers import AutoConfig, AutoTokenizer, AutoModel, TFAutoModel
from transformers import pipeline
from hazm import *
import fasttext
from hazm.utils import stopwords_list

E:\PMChatBot\Bot\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from tqdm import tqdm
import emojis

### Importing Dataset

In [ ]:
df =pd.read_csv("MeDiaPQ&A/MeDiaPQA.csv", sep=';')

In [ ]:
df.rename(columns={'Q':'question', 'A':'answer'}, inplace=True)

In [ ]:
df.head()

### Preprocessing Dataset

In [ ]:
def _multiple_replace(mapping, text):
    pattern = "|".join(map(re.escape, mapping.keys()))
    return re.sub(pattern, lambda m: mapping[m.group()], str(text))

def convert_fa_numbers(input_str):
    mapping = {
        '۰': '0',
        '۱': '1',
        '۲': '2',
        '۳': '3',
        '۴': '4',
        '۵': '5',
        '۶': '6',
        '۷': '7',
        '۸': '8',
        '۹': '9',
        '.': '.',
    }
    return _multiple_replace(mapping, input_str)


def convert_ar_characters(input_str):
    """
    Converts Arabic chars to related Persian unicode char
    :param input_str: String contains Arabic chars
    :return: New str with converted arabic chars
    """
    mapping = {
        'ك': 'ک',
        'ى': 'ی',
        'ي': 'ی',
        'ئ':'ی',
        'إ':'ا',
        'أ':'ا',
        'ة':'ه',
        'ؤ':'و'
    }
    return _multiple_replace(mapping, input_str)


def preprocess(text):
    #extractor = URLExtract()
    #for url in extractor.gen_urls(text):
    #    text = text.replace(url,'<URL>')
    emj = emojis.get(text)
    #emj = emoji
    for i in emj:
        if i in text:
            text = text.replace(i,'<emoji>')
    text = convert_fa_numbers(text)
    text = convert_ar_characters(text)
    # regex to detect and replace all smilies in the text with <smiley>
    text = re.sub(r"(:\s?\)|:-\)|\(\s?:|\(-:|:\'\)|:\s?D|8-\)|:\s?\||;\s?\)|:-\*|:-\||:-\(|:\s?P|:-P|:-p|:-b|:-O|:-o|:-0|:-\@|:\$|:-\^|:-&|:-\*|:-\+|:-\~|:-\`|:-\>|:-\<|:-\}|:-\{|\[:\s?\]|\[:\s?\]|:\s?\]|:\s?\[|:\s?\}|:\s?\{)",'<smiley>',text)
    text = text.lower() # we lowercase here to prevent changes in the URLs and smilies
    text = text.strip()
    text = re.sub(r'[<>#.:()"\'!?؟،,@$%^&*_+\[\]/]', ' ', text)
    text = re.sub(r'[\s]{2,}', ' ', text)
    text = re.sub(r'(\w)\1{2,}', r'\1',text)
    if re.search(r'[\u0600-\u06FF]', text):
        return(text)
    else:
        return 'None'

In [ ]:
tqdm.pandas()

In [ ]:
df['questions'] = df['question'].progress_apply(preprocess)
df['answers'] = df['answer'].progress_apply(preprocess)

In [ ]:
df.head()

In [ ]:
df.drop(['question', 'answer'],axis=1, inplace=True)

In [ ]:
df

In [ ]:
df = df.dropna()

In [ ]:
df.info()

In [ ]:
#df.to_csv('PMChatbot_Datasets_new.csv', index=False, encoding='utf-8-sig')

In [ ]:
#stop_words = stopwords_list()
#df['Questions'] = df['questions'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
#df['Answers'] = df['answers'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

In [ ]:
#df.head()

### NEW PROCESSING

In [ ]:
df =pd.read_csv("PMChatbot_Datasets_new.csv", sep=',')

In [ ]:
df.rename(columns={'questions':'QQ', 'answers':'AA'}, inplace=True)

In [ ]:
df

In [ ]:
df['QQ'] = df['QQ'].str.replace("\u200c", " ")
df['AA'] = df['AA'].str.replace("\u200c", " ")

In [ ]:
#df.to_csv('PMChatbot_Datasets_BESTnew.csv', index=False, encoding='utf-8-sig')

In [ ]:
import pandas as pd

# خواندن فایل اکسل
df = pd.read_csv("PMChatbot_Datasets_BESTnew.csv")

In [ ]:
from hazm import WordTokenizer

# ایجاد یک شیء برای توکن‌سازی کلمات
tokenizer = WordTokenizer()

# تابع برای فاصله‌گذاری بین کلمات
def add_spaces(text):
    tokens = tokenizer.tokenize(text)
    return " ".join(tokens)

# اعمال تابع روی ستون مورد نظر
df["QQ"] = df["QQ"].apply(add_spaces)
df["AA"] = df["AA"].apply(add_spaces)

In [ ]:
print(df["QQ"][3])

In [ ]:
from hazm import WordTokenizer

# ایجاد یک شیء برای توکن‌سازی کلمات
tokenizer = WordTokenizer()

# تابع برای فاصله‌گذاری بین کلمات
def add_spaces(text):
    tokens = tokenizer.tokenize(text)
    return " ".join(tokens)

# اعمال تابع روی ستون مورد نظر
#df = df.apply(add_spaces)

In [ ]:
df = ["نمیادبایدچیکارکنم"].apply(add_spaces)

In [ ]:
import pandas as pd
from hazm import WordTokenizer

# ایجاد یک DataFrame از لیست
df = pd.DataFrame(["نمیادبایدچیکارکنم"], columns=["متن"])

# ایجاد یک شیء برای توکن‌سازی کلمات
tokenizer = WordTokenizer()

# تابع برای فاصله‌گذاری بین کلمات
def add_spaces(text):
    tokens = tokenizer.tokenize(text)
    return " ".join(tokens)

# اعمال تابع روی ستون "متن"
df["متن"] = df["متن"].apply(add_spaces)

# نمایش نتیجه
print(df)

### ENDER

In [3]:
df = pd.read_csv("PMChatbot_Datasets_BESTnew.csv")

In [4]:
import re

def remove_noise(text):
    # حذف اموجی‌ها و کاراکترهای خاص
    text = re.sub(r'[^\w\s\u200c.,،؛!؟?]', '', text)
    # حذف فاصله‌های اضافی
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [5]:
df['QQ'] = df['QQ'].apply(remove_noise)
df['AA'] = df['AA'].apply(remove_noise)

In [6]:
from hazm import Normalizer

normalizer = Normalizer()

df['QQ'] = df['QQ'].apply(normalizer.normalize)
df['AA'] = df['AA'].apply(normalizer.normalize)

In [7]:
#df['QQ'] = df['QQ'].str.replace("\u200c", " ")
#df['AA'] = df['AA'].str.replace("\u200c", " ")

In [7]:
def separate_specific_words(text):
    # Dictionary for specific words, including the new separation
    word_dict = {
        "دکترپسرم":"دکتر پسرم",
        "پدربنده":"پدر بنده",
        "دکترایشان":"دکتر ایشان",
        "باردارنمی":"باردار نمی",
        "افسرده_شدم":"افسرده شدم",
        "داشته_باشد":"داشته باشد",
        "داده_بودم":"داده بودم",
        "گرفته_بودم":"گرفته بودم",
        "انگارکه":"انگار که",
        "شده_اند":"شده اند",
        "افتاده_است":"افتاده است",
        "دکترحدود":"دکتر حدود",
        "بهتراست":"بهتر است",
        "دارمحتی":"دارم حتی",
        "کاربکنیم":"کار بکنیم",
        "بارداری":"بار داری",
        "وسینورا":"و سینورا",
        "اثرنکرد":"اثر نکرد",
        "بالابود":"بالا بود",
        "چهاربار":"چهار بار",
        "شده_بود":" شده بود",
        "شهریورماه":"شهریور ماه",
        "دکتراجازه":"دکتر اجازه",
        "ازامروز":"از امروز",
        "بیشتراز":"بیشتر از",
        "شدپرستارمجددا":"شد پرستار مجددا",
        "ودراوردن":"و در اوردن",
        "وکشیده":"و کشیده",
        "بودموقع":"بود موقع",
        "دوهفته":"دو هفته",
        "نمیادباید":"نمی‌آید باید",
        "نمی‌آیدباید": "نمی‌آید باید",  # Separate "نمی‌آید" and "باید"
        "چیکارکنم": "چیکار کنم",       # Separate "چیکار" and "کنم"
        "بایدچیکار": "باید چیکار",     # Separate "باید" and "چیکار"  <-- Added this line
        
        # Add more words as needed...
    }

    # Replace joined words with separated versions
    for word, replacement in word_dict.items():
        text = text.replace(word, replacement)
    return text

# Example usage
#text = "نمی‌آیدبایدچیکار کنم"
#separated_text = separate_specific_words(df['QQ'])
#print(separated_text)  # Output: نمی‌آید باید چیکار کنم

In [43]:
df['QQ'] = df['QQ'].apply(separate_specific_words)
df['AA'] = df['AA'].apply(separate_specific_words)

In [8]:
from hazm import word_tokenize

def find_joined_words(text):
    # توکن‌سازی متن
    tokens = word_tokenize(text)
    
    # شناسایی کلمات بهم چسبیده
    joined_words = [token for token in tokens if len(token) > 6]  # حدس طول کلمات بهم چسبیده
    return joined_words

# اعمال تابع روی ستون متن
df['کلمات بهم چسبیده سوال'] = separate_specific_words(df['QQ'].apply(find_joined_words))
df['کلمات بهم چسبیده جواب'] = separate_specific_words(df['AA'].apply(find_joined_words))

# نمایش داده‌های جدید
print(df.head())

                                                  QQ  \
0                                          سلام دکتر   
1  من معاینه شدم ولی چیزی نگفتن فقط گفتن سونو اون...   
2                                     سلام اقای دکتر   
3  دوهفته پیش بابام یک شب بیمارستان بستری بودموقع...   
4                               سلام خوبین آقای دکتر   

                                           AA  \
0                                  سلام بفرما   
1            خیر ازمایشات هورمونی انجامی دهید   
2                               سلام بفرمایید   
3  در این مورد به متخصص جراح کلیه مراجعه کنید   
4                               مرسی بفرمایید   

                               کلمات بهم چسبیده سوال کلمات بهم چسبیده جواب  
0                                                 []                    []  
1                                                 []   [ازمایشات, هورمونی]  
2                                                 []            [بفرمایید]  
3  [بیمارستان, بودموقع, دستشویی, ودراوردن, شدپرست...        

In [12]:
print(df['QQ'][230])
print(df['AA'][230])

دکترپسرم سه سالشه هنوز نمیتونه صحبت کنه صداش که می‌زنیم اهمیت نمیده و فقط به صداهای اطراف مانند تلویزیون اهمیت میده چیزی ازحرفاش هم خوب نمیشه فهمید فقط بابا یا گاهی هم مامان میگه هنوز چیزی میخواد اسمشو نمیگه چیکار کنم
در اولین فرصت تست شنوایی‌ای بی آر رو انجام بدید


In [12]:
#df.drop(['AA'],axis=1, inplace=True)

In [13]:
df.to_csv('end_output_file.csv', index=False, encoding='utf-8-sig')

print("نتایج با موفقیت ذخیره شدند!")

نتایج با موفقیت ذخیره شدند!


In [46]:
from hazm import word_tokenize, sent_tokenize
words = word_tokenize(separate_specific_words(df['QQ'][230]))
sentences = sent_tokenize(separate_specific_words(df['QQ'][230]))

In [47]:
print(separate_specific_words(df['QQ'][3]))

دو هفته پیش بابام یک شب بیمارستان بستری بود موقع دستشویی سنت و کشیده و در اوردن که خون زیادی اومد بعدش قطع شد پرستار مجددا سنت را وصل کرد گفت با ادرار اگه خون بیاد باید عمل بشه که خوشبختانه خون نیامد بیشتر از دو هفته مشکلی نداشت اما از امروز صبح ادرارش نمی‌آید باید چیکار کنم


In [40]:
print("جملات:", sentences)

جملات: ['سلام دکتر از زحمت\u200cهای ما']


In [41]:
print("کلمات:", words)

کلمات: ['دکترپسرم', 'سه', 'سالشه', 'هنوز', 'نمیتونه', 'صحبت', 'کنه', 'صداش', 'که', 'می\u200cزنیم', 'اهمیت', 'نمیده', 'و', 'فقط', 'به', 'صداهای', 'اطراف', 'مانند', 'تلویزیون', 'اهمیت', 'میده', 'چیزی', 'ازحرفاش', 'هم', 'خوب', 'نمیشه', 'فهمید', 'فقط', 'بابا', 'یا', 'گاهی', 'هم', 'مامان', 'میگه', 'هنوز', 'چیزی', 'میخواد', 'اسمشو', 'نمیگه', 'چیکار', 'کنم']
